# Taxi driver V3
QLearning algorith

By HERTZ Arthur, LUCAS Michaël, TROUSSET Alexis, ROUSSEL Rémi, BRILLARD Quentin.

## 1. What is [Taxi V3](https://gym.openai.com/envs/Taxi-v3/)

Taxi V3 is  gym environment that simulate A taxi inside a city. The environment can be illutrate by the following grid wich can be shown by the code bellow.


In [3]:
import gym
env = gym.make("Taxi-v3").env
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



### 1.1 Grid explanation
This grid above represent the city. 
On it we have for points, that are shown by letters (R,G,Y,B) that represent locations where we can pickup clients and drop them off. Each '|' represent wall taht the taxi can't pass through. The yellow rectangle represent the taxi, this rectangle is yellow when the taxi is free and green is when we have a client.

### 1.2 Reward notion
This environment give us a reward after each action that we send to gym. We have three possible reward:
- -1 pts Is for each action that does'nt have good or bad result
- -10 pts Is for each bad actions (try to pick up a client while we cant, or drop him off while we aren't on a good location)
- +20 pts For each successful drop-off

### 1.3 Action list

To resolve this "game" we can make six different actions:
- move south
- move north
- move est
- move west
- pickup
- dropdown

## 2 Reinforcement Learning

In reinforcement learning the agent encoured a state. With this state the agent have to takes actions according to the state. 
So to summarize our agent for a given state, for an action it obtains a certain result and we have to keep in memory each result from each actions from each states. So to keep all the result in memory we have to build a table.

### 2.1 Build the QTable

To keep track of all the results we need to build a table that summarizes the different actions on the x-axis (here the directional shifts and the action of picking up and dropping off a customer) and on the y-axis the states of the Taxi, i.e. where it is in the city.



In [5]:
import numpy
q_table = numpy.zeros([env.observation_space.n, env.action_space.n])

After init the qtable, we have to define all Hyperparameters:
- the alpha, the learning rate
- the gamma, the discount rate
- the epsilon, define the chance to select a random action (exploration) insteadof maximising reward (exploitation) 

In [6]:
alpha = 0.1  # Learning Rate
gamma = 0.6  # Discount Rate
epsilon = 0.1  # Chance of selecting a random action instead of maximising reward.

### 2.2 Exploration and exploitation

At the beginig we must know more about the environment (what reward we will get from an action made from a given state), to fil our qlearning table. A little at a time it is preferable to exploit all results to get best rewards. So the goal from the espilon variable is to at the begining maximized chance to do exploration and at the end maximzed chance to do exploitation. 

## 3 Lets starts !

### 3.1 Reset the environment

After import and select the good gym environment (CF part 1), we have to reset the environment, define variable to get some metrics and define number of episodes for training and for display

In [7]:
env.reset() # reset environment to a new, random state

# For plotting metrics
all_epochs = []
all_penalties = []

training_episodes = 20000  # Amount of times to run environment while training.
display_episodes = 10  # Amount of times to run environment after training.


 ### 3.2 Training the agent
  Now we are in the training step. The purpose from this part is to train our agent to take best descision in the future.
 

In [8]:
import random

for i in range(training_episodes):
    
    # Get the active state
    state = env.reset()
    # Define that we doesn't finished the episode
    done = False
    # Set penalities and rewards to 0
    penalties, reward = 0, 0
    
    # loop while we doesn't finish the current episode
    while not done:
        # Make a randome test if we have to do exploitation or exploration
        if random.uniform(0,1) < epsilon:
            # Make an exploration action
            action = env.action_space.sample()
        else:
            action = numpy.argmax(q_table[state])
        
        # Execute the next step
        next_state, reward, done, info = env.step(action)
        
        # Retrieve old value from the q-table.
        old_value = q_table[state, action]
        next_max = numpy.max(q_table[next_state])
        
        # Update q-value for the currentstate
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value
        
        # Check if we get a penalties
        if reward == -10:
            # Increment penalties number
            penalties += 1
        
        state = next_state
        
print("Training finished.\n")
    
    

Training finished.



### 3.3 Solve the game
After training the agent we are able to appy an algorithme to solve the game. But first we have to define two varaibles to get somme statistics about the algorithm execution. It is the total epochs and total penalties.

In [9]:
total_epochs, total_penalties = 0, 0

Now we are able toloop and display result for the following algorithme.

In [11]:
frames = [] # for animation
for _ in range(display_episodes):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0

    done = False

    while not done:
        action = numpy.argmax(q_table[state])
        state, reward, done, info = env.step(action)

        if reward == -10:
            penalties += 1

        epochs += 1
        
        # Put each rendered frame into dict for animation
        frames.append({
            'frame': env.render(mode='ansi'),
            'state': state,
            'action': action,
            'reward': reward
            }
        )

    total_penalties += penalties
    total_epochs += epochs

from IPython.display import clear_output
from time import sleep

def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.1)
        
print_frames(frames)


+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 130
State: 410
Action: 5
Reward: 20


## 3.4 Get the result

Now after algorithme execution we are able to display the final result.

In [15]:
print(f"Results after {display_episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / display_episodes}")
print(f"Average penalties per episode: {total_penalties / display_episodes}")

Results after 10 episodes:
Average timesteps per episode: 13.0
Average penalties per episode: 0.0
